In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext sql

In [2]:
%sql sqlite:///results.db

# Color

In [64]:
%%sql
create view aliases_colorizing_output as
select distinct alias_id
from alias join command using (alias_id)
where 
    (alias.value like '%color%' and alias.value not like '%color=never%')
    or (alias.value like '%colour%' and alias.value not like '%colour=never%')
    or alias.value like '%pygmentize%'
    or alias.value like '%grc %'
    or alias.value like '%grcat %'
    or (command.name = 'less' and command.arguments like '%r%')
    or (command.name = 'tmux' and (command.arguments like '%-2%' or command.arguments like '%-T 256%'))

 * sqlite:///results.db
Done.


[]

In [67]:
%%sql
with 
    total as (select count(*) as total from alias),
    total_redefs as (select count(distinct alias_id) as total_redefs from alias join command using (alias_id) where alias.name = command.name)
select 
    count(distinct alias_id) as '#', 
    round(count(distinct alias_id)*100.0/total,2) as '%',
    round(count(distinct alias_id)*100.0/total_redefs,2) as '% of redefines'
from alias 
join aliases_colorizing_output using (alias_id)
join command using (alias_id)
join total
join total_redefs
where (
    alias.name = command.name 
    or alias.name in known_commands
)

 * sqlite:///results.db
Done.


#,%,% of redefines
182624,8.29,57.21


In [68]:
%%sql
with total as (select count(*) as total from alias)
select 
    count(distinct alias_id) as '#', 
    round(count(distinct alias_id)*100.0/total,2) as '%'
from alias
join aliases_colorizing_output using (alias_id)
join total

 * sqlite:///results.db
Done.


#,%
223698,10.15


---
## By Command

In [82]:
%%sql
select 
    cmd_name,
    count(distinct alias_id) as num,
    round(count(distinct alias_id)*100.0/total,2) as per
from (
    select command.name as cmd_name, alias_id
    from alias 
    join aliases_colorizing_output using (alias_id) 
    join command using (alias_id)
    
    union
    
    select alias.name as cmd_name, alias_id
    from alias
    join aliases_colorizing_output using (alias_id)
    where alias.name in known_commands
)   
join command_alias_totals using (cmd_name)
group by cmd_name
having num > 1000 and per > 20
order by per desc, num desc

 * sqlite:///results.db
Done.


cmd_name,num,per
pygmentize,1410,100.0
TERM=xterm-256color,1228,100.0
TERM=screen-256color-bce,1076,100.0
colourify,1599,99.94
colorls,1188,99.92
colordiff,2143,99.91
grc,1997,99.75
fgrep,26899,98.58
vdir,3155,95.58
egrep,28224,90.48
